In [1]:
using CUDA, BenchmarkTools

In [2]:
CUDA.memory_status() 

Effective GPU memory usage: 4.69% (379.938 MiB/7.915 GiB)
CUDA allocator usage: 0 bytes
binned usage: 0 bytes (0 bytes allocated, 0 bytes cached)


In [3]:
cpu = rand(Float32, floor(Int, (10^9)/4))
size = Base.summarysize(cpu)

1000000040

In [4]:
gpu = CuArray{Float32}(undef, floor(Int, (10^9)/4));

In [5]:
CUDA.memory_status() 

Effective GPU memory usage: 16.46% (1.303 GiB/7.915 GiB)
CUDA allocator usage: 953.674 MiB
binned usage: 16 bytes (16 bytes allocated, 0 bytes cached)
Discrepancy of 953.674 MiB between memory pool and allocator!


In [6]:
@btime copyto!(gpu, cpu);

  105.030 ms (0 allocations: 0 bytes)


In [7]:
@btime copyto!(cpu, gpu);

  112.806 ms (0 allocations: 0 bytes)


In [ ]:
floor(Int, (10^9)/4)

In [ ]:
?trunc

In [ ]:
VERSION, Base.Threads.nthreads(), Base.Sys.CPU_THREADS

In [ ]:
using CUDA, BenchmarkTools, Humanize
import Humanize: digitsep

In [ ]:
@assert CUDA.functional(true)

In [ ]:
CUDA.memory_status() 

In [ ]:
N = 10^9
digitsep(N)

In [ ]:
cpu_a = rand(Float32, N);

In [ ]:
digitsep(Base.summarysize(cpu_a))

In [ ]:
cpu_b = Array{Float32}(undef, N);

In [ ]:
CUDA.memory_status() 

In [ ]:
4 / 0.42

In [ ]:
@benchmark cpu2 = Array(gpu)

In [ ]:
@benchmark copyto!(cpu_b, gpu)

In [ ]:
@benchmark copyto!(cpu_b, cpu_a)

In [ ]:
@btime copyto!(cpu_b, cpu_a);

In [ ]:
4 / 0.3

In [ ]:
?copyto!

In [ ]:
?Array

In [ ]:
include("sorting.jl")

In [ ]:
using DataStructures, Random, Test

In [ ]:
function test_quicksort(T, f, N)
    a = map(x -> T(f(x)), 1:N)
    c = CuArray(a)
    quicksort!(c)
    @test Array(c) == sort(a)
end

In [ ]:
test_quicksort(Float32, x -> rand(Float32), 10000)

In [ ]:
N = 2^20
x = fill(1.0f0, N)  # a vector filled with 1.0 (Float32)
y = fill(2.0f0, N)  # a vector filled with 2.0


In [ ]:
using Test
@test all(y .== 3.0f0)

In [ ]:
function sequential_add_A!(y, x)
    for i in eachindex(y, x)
        @inbounds y[i] += x[i]
    end
    return nothing
end

In [ ]:
function sequential_add_B!(y, x)
    for i in eachindex(y, x)
        y[i] += x[i]
    end
    return nothing
end

In [ ]:
x = fill(1.0f0, N)  # a vector filled with 1.0 (Float32)
y = fill(2.0f0, N)  # a vector filled with 2.0
@benchmark sequential_add_A!(y, x)

In [ ]:
x = fill(1.0f0, N)  # a vector filled with 1.0 (Float32)
y = fill(2.0f0, N)  # a vector filled with 2.0
@benchmark sequential_add_B!(y, x)

In [ ]:
function parallel_add_A!(y, x)
    Threads.@threads for i in eachindex(y, x)
        @inbounds y[i] += x[i]
    end
    return nothing
end

In [ ]:
function parallel_add_B!(y, x)
    Threads.@threads for i in eachindex(y, x)
        y[i] += x[i]
    end
    return nothing
end

In [ ]:
@benchmark parallel_add_A!(y, x)

In [ ]:
@benchmark parallel_add_B!(y, x)